In [8]:
from keras.models import load_model

def load_and_view_model(file_path):
    # Load the model
    model = load_model(file_path)

    # Print the model summary
    model.summary()

    return model

model_file_path = 'models/bd_net.h5'
model = load_and_view_model(model_file_path)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input (InputLayer)          [(None, 55, 47, 3)]          0         []                            
                                                                                                  
 conv_1 (Conv2D)             (None, 52, 44, 20)           980       ['input[0][0]']               
                                                                                                  
 pool_1 (MaxPooling2D)       (None, 26, 22, 20)           0         ['conv_1[0][0]']              
                                                                                                  
 conv_2 (Conv2D)             (None, 24, 20, 40)           7240      ['pool_1[0][0]']              
                                                                                            

In [4]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import keras
import sys
import h5py
import warnings
from tqdm import tqdm

2023-12-04 21:24:50.437610: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-04 21:24:53.317478: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-04 21:24:53.317570: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-04 21:24:53.634822: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-04 21:24:54.238342: I tensorflow/core/platform/cpu_feature_guar

In [5]:
clean_valid_data_dir = 'data/cl/valid.h5'
poisoned_valid_data_dir = 'data/bd/bd_valid.h5'
model_dir = 'models/bd_net.h5'

def data_loader(filepath):
    data = h5py.File(filepath, 'r')
    x_data = np.array(data['data'])
    y_data = np.array(data['label'])
    x_data = x_data.transpose((0,2,3,1))

    return x_data, y_data

def accuracy_calculator(x_test, y_test, model):
    predicted_labels = np.argmax(model.predict(x_test), axis=1)
    accuracy = np.mean(np.equal(predicted_labels, y_test)) * 100
    return accuracy

def calculate_attack_success_rate(poisoned_x_test, poisoned_y_test, model):
    poisoned_predictions = np.argmax(model.predict(poisoned_x_test), axis=1)
    # ASR is the proportion of poisoned samples that are misclassified
    asr = np.mean(poisoned_predictions == poisoned_y_test) * 100
    return asr

def main():
    clean_x_valid, clean_y_valid = data_loader(clean_valid_data_dir)
    bd_x_valid, bd_y_valid = data_loader(poisoned_valid_data_dir)

    bd_model = keras.models.load_model(model_dir)

    
    clean_accuracy = accuracy_calculator(clean_x_valid, clean_y_valid, bd_model)
    print('Clean Classification accuracy:', clean_accuracy)
    
    poisoned_accuracy = accuracy_calculator(bd_x_valid, bd_y_valid, bd_model)
    print('Poisoned Classification accuracy:', poisoned_accuracy)
    

if __name__ == '__main__':
    main()

2023-12-04 21:25:05.228552: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31141 MB memory:  -> device: 0, name: Tesla V100-PCIE-32GB, pci bus id: 0000:86:00.0, compute capability: 7.0
2023-12-04 21:25:07.437060: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904


361/361 [==============================] - 5s 2ms/step
Clean Classification accuracy: 98.64899974019225
361/361 [==============================] - 1s 2ms/step
Poisoned Classification accuracy: 100.0


In [6]:
model = keras.models.load_model(model_dir)
clean_x_valid, clean_y_valid = data_loader(clean_valid_data_dir)
bd_x_valid, bd_y_valid = data_loader(poisoned_valid_data_dir)


def prune_channel(model, layer_name, channel_index):
    layer = model.get_layer(layer_name)
    next_layer = model.get_layer('conv_3')
    weights, biases = next_layer.get_weights()
    weights[:, :, :, channel_index] = 0  # Prune the channel
    biases[channel_index] = 0
    next_layer.set_weights([weights, biases])
    return model

def pruned_and_save(model, layer_name, clean_validation_data_x, clean_validation_data_y,poisoned_x_valid, poisoned_y_valid):
    original_accuracy = accuracy_calculator(clean_validation_data_x, clean_validation_data_y, model)
    saved_models = [ False, False, False]
    model_dummy = keras.models.clone_model(model)
    model_dummy.set_weights(model.get_weights())  # Copy weights from the original model

    modelinput = model.input
    conv3output = model.get_layer(layer_name).output
    partial_model = keras.models.Model(inputs=modelinput, outputs=conv3output)

    # Compute average activations
    conv3output_values = partial_model.predict(clean_validation_data_x)
    avg_activations = np.mean(conv3output_values, axis=(0, 1, 2))
    # Sort channels by activation in ascending order as mentioned in Prof Garg's Paper
    channels_sorted = np.argsort(avg_activations)# descending order activations did not yield any results so I used the paper approach

    
        
    

    channels_pruned = 0 # Tracking number of pruned channels
    for channel in channels_sorted:
            # Pruning the channel
            model_dummy = prune_channel(model_dummy, layer_name, channel)
            channels_pruned += 1

            # Measuring new validation accuracy
            new_accuracy = accuracy_calculator(clean_validation_data_x, clean_validation_data_y, model_dummy)
            new_asr = calculate_attack_success_rate(poisoned_x_valid, poisoned_y_valid, model_dummy)
            print(f"Pruning {channel} channel, Current Accuracy: {new_accuracy} ,ASR: {new_asr},Channel number: {channels_pruned}")

            if new_accuracy <= original_accuracy - 2 and not saved_models[0]:
                model_filename = 'models/model_pruned__drop2.h5'
                model_dummy.save(model_filename)
                saved_models[0] = True
                print(f"Model saved as {model_filename}. Stopped at {2}% drop in accuracy after pruning {channels_pruned} channels.")
                
                
            if new_accuracy <= original_accuracy - 4 and not saved_models[1]:
                model_filename = 'models/model_pruned__drop4.h5'
                model_dummy.save(model_filename)
                saved_models[1] = True
                print(f"Model saved as {model_filename}. Stopped at {4}% drop in accuracy after pruning {channels_pruned} channels.")
                
                
            if new_accuracy <= original_accuracy - 10 and not saved_models[2]:
                model_filename = 'models/model_pruned__drop10.h5'
                model_dummy.save(model_filename)
                saved_models[2] = True
                print(f"Model saved as {model_filename}. Stopped at {10}% drop in accuracy after pruning {channels_pruned} channels.")
                
            if all(saved_models):
                break
                

    return saved_models




saved_model_filenames = pruned_and_save(model, 'conv_3', clean_x_valid,clean_y_valid,bd_x_valid, bd_y_valid)



361/361 [==============================] - 1s 2ms/step
Pruning 0 channel, Current Accuracy: 98.64899974019225 ,ASR: 100.0,Channel number: 1
361/361 [==============================] - 1s 1ms/step
Pruning 26 channel, Current Accuracy: 98.64899974019225 ,ASR: 100.0,Channel number: 2
361/361 [==============================] - 1s 1ms/step
Pruning 27 channel, Current Accuracy: 98.64899974019225 ,ASR: 100.0,Channel number: 3
361/361 [==============================] - 1s 1ms/step
Pruning 30 channel, Current Accuracy: 98.64899974019225 ,ASR: 100.0,Channel number: 4
361/361 [==============================] - 1s 1ms/step
Pruning 31 channel, Current Accuracy: 98.64899974019225 ,ASR: 100.0,Channel number: 5
361/361 [==============================] - 1s 2ms/step
Pruning 33 channel, Current Accuracy: 98.64899974019225 ,ASR: 100.0,Channel number: 6
361/361 [==============================] - 1s 2ms/step
Pruning 34 channel, Current Accuracy: 98.64899974019225 ,ASR: 100.0,Channel number: 7
361/361 [=====

/home/ro2151/.local/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


361/361 [==============================] - 1s 2ms/step
Pruning 16 channel, Current Accuracy: 95.20221702606739 ,ASR: 99.9913397419243,Channel number: 46
361/361 [==============================] - 1s 2ms/step
Pruning 5 channel, Current Accuracy: 94.77786438035854 ,ASR: 100.0,Channel number: 47
361/361 [==============================] - 1s 2ms/step
Pruning 56 channel, Current Accuracy: 94.344851476574 ,ASR: 99.9913397419243,Channel number: 48
Model saved as models/model_pruned__drop4.h5. Stopped at 4% drop in accuracy after pruning 48 channels.
361/361 [==============================] - 1s 2ms/step
Pruning 8 channel, Current Accuracy: 93.85121676625964 ,ASR: 99.98267948384861,Channel number: 49
361/361 [==============================] - 1s 2ms/step
Pruning 11 channel, Current Accuracy: 92.16246644149996 ,ASR: 80.47977829739327,Channel number: 50
361/361 [==============================] - 1s 2ms/step
Pruning 46 channel, Current Accuracy: 89.17467740538669 ,ASR: 80.73958603966398,Channel n

In [7]:
# Loading models
bad_net = keras.models.load_model('models/bd_net.h5')
model_pruned_drop2 = keras.models.load_model('models/model_pruned__drop2.h5')
model_pruned_drop4 = keras.models.load_model('models/model_pruned__drop4.h5')
model_pruned_drop10 = keras.models.load_model('models/model_pruned__drop10.h5')

clean_test_data_dir = 'data/cl/valid.h5'
poisoned_test_data_dir = 'data/bd/bd_valid.h5'
# Preparing test data and labels for both clean and poisoned data
clean_x_test, clean_y_test = data_loader(clean_test_data_dir)
bd_x_test, bd_y_test = data_loader(poisoned_test_data_dir) 

def calculate_accuracy(predictions, true_labels):
    correct_predictions = np.equal(predictions, true_labels)
    accuracy = np.mean(correct_predictions) * 100
    return accuracy

def combined_model_prediction(model1, model2, data):
    predictions1 = np.argmax(model1.predict(data), axis=1)
    predictions2 = np.argmax(model2.predict(data), axis=1)

    combined_predictions = np.where(predictions1 == predictions2, predictions1, 1283)
    return combined_predictions


In [13]:
def evaluate_combined_models(base_model, pruned_models, clean_x, clean_y, bd_x, bd_y):
    results = {}
    for model_name, pruned_model in pruned_models.items():
        # Combined predictions on clean test data
        combined_clean_preds = combined_model_prediction(base_model, pruned_model, clean_x)#G Goodnet Model
        clean_test_acc = calculate_accuracy(combined_clean_preds, clean_y)
        
        # Combined predictions on poisoned data
        combined_bd_preds = combined_model_prediction(base_model, pruned_model, bd_x)#G Goodnet Model
        bd_asr = calculate_accuracy(combined_bd_preds, bd_y)
        
        results[model_name] = {
            'clean_test_accuracy': clean_test_acc,
            'bad_asr': bd_asr
        }

        print(f'For {model_name}: Clean Test Data Accuracy: {clean_test_acc:.2f}%, Attack Success Rate: {bd_asr:.2f}%')

    return results


# Dictionary of pruned models
pruned_models = {
    'GoodNetdrop2': model_pruned_drop2,
    'GoodNetdrop4': model_pruned_drop4,
    'GoodNetdrop10': model_pruned_drop10
}

# Evaluating combined models
evaluation_results = evaluate_combined_models(bad_net, pruned_models, clean_x_test, clean_y_test, bd_x_test, bd_y_test)


361/361 [==============================] - 1s 2ms/step
For GoodNetdrop2: Clean Test Data Accuracy: 95.62%, Attack Success Rate: 100.00%
361/361 [==============================] - 1s 2ms/step
For GoodNetdrop4: Clean Test Data Accuracy: 94.12%, Attack Success Rate: 99.99%
361/361 [==============================] - 1s 1ms/step
For GoodNetdrop10: Clean Test Data Accuracy: 84.25%, Attack Success Rate: 77.02%
